# Install additional Python libraries

In [1]:
!pip install quantities

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Delete existing DB

In [2]:
!rm simoc_server/db.sqlite

# Import Python libraries

In [3]:
import json
from random import randint

In [4]:
import pprint
pp = pprint.PrettyPrinter()

In [5]:
from simoc_server import app, db
from simoc_server.database.db_model import User
from simoc_server.database.seed_data import seed
from simoc_server.game_runner import GameRunnerManager, GameRunnerInitializationParams

Unknown DB_TYPE variable: "None"
Using SQLite by default


# Load Agent model

In [6]:
db.drop_all()
db.create_all()
seed.seed('agent_desc.json')

# Create new user

In [7]:
username = 'user1'
password = 'user1'

user = User.query.filter_by(username=username).first()
if not user:
    print('Creating new user')
    user = User(username=username)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()

Creating new user


# Initialize the ABM

In [8]:
game_config = {"agents": {
    "human_agent": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                     "power_storage": [1], "food_storage": [1]}, "amount": 2}],
    "cabbage": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "lettuce": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "greenhouse_medium": [{"connections": {"power_storage": [1]}, "amount": 1}],
    "solar_pv_array": [{"connections": {"power_storage": [1]}, "amount": 100}],
    "multifiltration_purifier_post_treament": [{"connections": {"water_storage": [1, 2]}, "amount": 1}],
    "urine_recycling_processor_VCD": [{"connections": {"power_storage": [1], "water_storage": [1, 2]}, "amount": 1}]},
"storages": {
    "air_storage": [{"id": 1, "atmo_h2o": 100, "atmo_o2": 100, "atmo_co2": 100}],
    "water_storage": [{"id": 1, "h2o_potb": 100, "h2o_tret": 100}, {"id": 2, "h2o_wste": 100, "h2o_urin": 100}],
    "nutrient_storage": [{"id": 1, "sold_n": 100, "sold_p": 100, "sold_k": 100, "sold_wast": 0}],
    "power_storage": [{"id": 1, "enrg_kwh": 1000, "heat_cal": 1000}],
    "food_storage": [{"id": 1, "food_edbl": 200}]},
"termination": {
    "time": {"value": 5, "unit": "hour"}},
    "food_leaf": {"value": 10000, "unit": "kg"},
    "evacuation": {}
}

game_runner_manager = GameRunnerManager()
game_runner_init_params = GameRunnerInitializationParams(game_config)
game_runner_manager.new_game(user, game_runner_init_params)
game_runner = game_runner_manager.get_game_runner(user)

In [9]:
game_runner.agent_model.get_total_storages()

{'total_currencies': {'atmo_o2': {'value': 100,
   'capacity': 400,
   'units': 'kg'},
  'atmo_co2': {'value': 100, 'capacity': 200, 'units': 'kg'},
  'atmo_n2': {'value': 0, 'capacity': 400, 'units': 'kg'},
  'atmo_ch4': {'value': 0, 'capacity': 200, 'units': 'kg'},
  'atmo_h2': {'value': 0, 'capacity': 100, 'units': 'kg'},
  'atmo_h2o': {'value': 100, 'capacity': 200, 'units': 'kg'},
  'h2o_treated': {'value': 0, 'capacity': 200, 'units': 'kg'},
  'h2o_potb': {'value': 100, 'capacity': 800, 'units': 'kg'},
  'h2o_urin': {'value': 100, 'capacity': 400, 'units': 'kg'},
  'h2o_wste': {'value': 100, 'capacity': 500, 'units': 'kg'},
  'h2o_tret': {'value': 100, 'capacity': 500, 'units': 'kg'},
  'sold_n': {'value': 100, 'capacity': 100, 'units': 'kg'},
  'sold_p': {'value': 100, 'capacity': 100, 'units': 'kg'},
  'sold_k': {'value': 100, 'capacity': 100, 'units': 'kg'},
  'sold_wast': {'value': 0, 'capacity': 100, 'units': 'kg'},
  'enrg_kwh': {'value': 1000, 'capacity': 1000, 'units': 'k

In [10]:
game_runner.agent_model.get_total_agents()

{'total_production': {'atmo_co2': {'value': 0.128, 'units': 'kg'},
  'atmo_h2o': {'value': 0.222, 'units': 'kg'},
  'h2o_wste': {'value': 0.12524999999999997, 'units': 'kg'},
  'sold_wast': {'value': 0.0675, 'units': 'kg'},
  'h2o_urin': {'value': 0.125, 'units': 'kg'},
  'heat_cal': {'value': 0.7, 'units': 'cal'},
  'atmo_o2': {'value': 5.739060121999999, 'units': 'g'},
  'food_edbl': {'value': 0.0, 'units': 'g'},
  'enrg_kwh': {'value': 30.00000000000005, 'units': 'kWh'},
  'h2o_potb': {'value': 4.75, 'units': 'kg'},
  'h2o_tret': {'value': 1.36, 'units': 'kg'}},
 'total_consumption': {'atmo_o2': {'value': 98.4, 'units': 'g'},
  'h2o_potb': {'value': 0.8536237500000333, 'units': 'kg'},
  'food_edbl': {'value': 199.5, 'units': 'g'},
  'heat_cal': {'value': 0.0, 'units': 'cal'},
  'atmo_co2': {'value': 0.007874524353000002, 'units': 'kg'},
  'sold_n': {'value': 0.20487643800000005, 'units': 'g'},
  'sold_p': {'value': 0.0053223559999999994, 'units': 'g'},
  'sold_k': {'value': 0.040110

# Run the simiulation

In [11]:
num_steps = 10
for i in range(1, num_steps + 1):
    agent_model_state = game_runner_manager.get_step(user, i)
#     if i % 100 == 0:
#         print("Step: {}".format(i))
#         pp.pprint(agent_model_state)
pp.pprint(agent_model_state)

{'agents': {'total_agent_types': {'cabbage': 10,
                                  'greenhouse_medium': 1,
                                  'human_agent': 2,
                                  'lettuce': 10,
                                  'multifiltration_purifier_post_treament': 1,
                                  'solar_pv_array': 100,
                                  'urine_recycling_processor_VCD': 1},
            'total_consumption': {'atmo_co2': {'units': 'kg',
                                               'value': 0.007874524353},
                                  'atmo_o2': {'units': 'g', 'value': 98.4},
                                  'enrg_kwh': {'units': 'kWh',
                                               'value': 3.2673245576666674},
                                  'food_edbl': {'units': 'g', 'value': 199.5},
                                  'h2o_potb': {'units': 'kg',
                                               'value': 0.8536237500000332},
                

In [12]:
game_runner_manager.save_all(allow_repeat_save=False)

In [13]:
game_runner.agent_model.get_total_storages()

{'total_currencies': {'food_edbl': {'value': 199.0025,
   'capacity': 1000,
   'units': 'kg'},
  'h2o_treated': {'value': 0, 'capacity': 200, 'units': 'kg'},
  'h2o_potb': {'value': 119.6949042500002, 'capacity': 800, 'units': 'kg'},
  'h2o_urin': {'value': 96.97499999999997, 'capacity': 400, 'units': 'kg'},
  'h2o_wste': {'value': 100.62624999999998, 'capacity': 500, 'units': 'kg'},
  'h2o_tret': {'value': 91.52500000000003, 'capacity': 500, 'units': 'kg'},
  'enrg_kwh': {'value': 999.9467350884665, 'capacity': 1000, 'units': 'kWh'},
  'heat_cal': {'value': 1000, 'capacity': 1000, 'units': 'cal'},
  'atmo_o2': {'value': 99.53669530061035, 'capacity': 400, 'units': 'kg'},
  'atmo_co2': {'value': 100.60062737823472, 'capacity': 200, 'units': 'kg'},
  'atmo_n2': {'value': 0, 'capacity': 400, 'units': 'kg'},
  'atmo_ch4': {'value': 0, 'capacity': 200, 'units': 'kg'},
  'atmo_h2': {'value': 0, 'capacity': 100, 'units': 'kg'},
  'atmo_h2o': {'value': 101.11000000000004, 'capacity': 200, 'un

In [14]:
game_runner.agent_model.get_total_agents()

{'total_production': {'h2o_tret': {'value': 1.36, 'units': 'kg'},
  'heat_cal': {'value': 0.7, 'units': 'cal'},
  'enrg_kwh': {'value': 30.00000000000005, 'units': 'kWh'},
  'h2o_potb': {'value': 4.75, 'units': 'kg'},
  'atmo_o2': {'value': 5.739060121999998, 'units': 'g'},
  'food_edbl': {'value': 0.0, 'units': 'g'},
  'atmo_co2': {'value': 0.128, 'units': 'kg'},
  'atmo_h2o': {'value': 0.222, 'units': 'kg'},
  'h2o_wste': {'value': 0.12524999999999997, 'units': 'kg'},
  'sold_wast': {'value': 0.0675, 'units': 'kg'},
  'h2o_urin': {'value': 0.125, 'units': 'kg'}},
 'total_consumption': {'h2o_urin': {'value': 1.36, 'units': 'kg'},
  'enrg_kwh': {'value': 3.2673245576666674, 'units': 'kWh'},
  'h2o_tret': {'value': 4.75, 'units': 'kg'},
  'atmo_co2': {'value': 0.007874524353, 'units': 'kg'},
  'h2o_potb': {'value': 0.8536237500000332, 'units': 'kg'},
  'sold_n': {'value': 0.20487643800000008, 'units': 'g'},
  'sold_p': {'value': 0.005322356, 'units': 'g'},
  'sold_k': {'value': 0.040110

In [15]:
game_runner.agent_model.get_model_stats()

{'step': 5,
 'is_terminated': True,
 'time': 21600.0,
 'agents': {'total_production': {'h2o_tret': {'value': 1.36, 'units': 'kg'},
   'heat_cal': {'value': 0.7, 'units': 'cal'},
   'enrg_kwh': {'value': 30.00000000000005, 'units': 'kWh'},
   'h2o_potb': {'value': 4.75, 'units': 'kg'},
   'atmo_o2': {'value': 5.739060121999998, 'units': 'g'},
   'food_edbl': {'value': 0.0, 'units': 'g'},
   'atmo_co2': {'value': 0.128, 'units': 'kg'},
   'atmo_h2o': {'value': 0.222, 'units': 'kg'},
   'h2o_wste': {'value': 0.12524999999999997, 'units': 'kg'},
   'sold_wast': {'value': 0.0675, 'units': 'kg'},
   'h2o_urin': {'value': 0.125, 'units': 'kg'}},
  'total_consumption': {'h2o_urin': {'value': 1.36, 'units': 'kg'},
   'enrg_kwh': {'value': 3.2673245576666674, 'units': 'kWh'},
   'h2o_tret': {'value': 4.75, 'units': 'kg'},
   'atmo_co2': {'value': 0.007874524353, 'units': 'kg'},
   'h2o_potb': {'value': 0.8536237500000332, 'units': 'kg'},
   'sold_n': {'value': 0.20487643800000008, 'units': 'g'},